In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 462 kB 49.5 MB/s 


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [4]:
df = pd.read_csv('hindi-train.csv')
df.head()

,headline,Label
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports
2,"SAI का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर म...",Sports
3,Pro Kabaddi League: ऑलराउंडर रोहित गूलिया के द...,Sports
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports


In [5]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [6]:
count_length()

In [7]:
df.head(10)

,headline,Label,word_count
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports,19
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports,17
2,"SAI का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर म...",Sports,18
3,Pro Kabaddi League: ऑलराउंडर रोहित गूलिया के द...,Sports,20
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports,13
5,Pro Kabaddi League: पटना पायरेट्स और बंगाल वॉर...,Sports,17
6,"I League: भारतीय फुटबॉल में कोरोना विस्फोट, AI...",Sports,17
7,'दो चैंपियन' के साथ ट्रेनिंग करते नजर आए शिखर ...,Sports,19
8,IND vs SA: जोहानिसबर्ग टेस्ट में पुजारा और रहा...,Sports,21
9,कप्तानी विवाद पर पूर्व पाक कप्तान सलमान बट ने ...,Sports,19


In [8]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords
stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्ह', 'तनह', 'तस', 'तस', 'त', 'थ', 'थ', 'थ', 'दबर', 'दय', 'दसर', 'दसर', 'द', 'दवर', 'न', 'नह', 'न', 'नहयत', 'नच', 'न', 'पर', 'पर', 'पहल', 'पर', 'प', 'फर', 'बन', 'बह', 'बहत', 'बद', 'बल', 'बलकल', 'भ', 'भतर', 'मगर', 'मन', 'म', 'म', 'यद', 'यह', 'यह', 'यह', 'य', 'यह', 'य', 'रख', 'रह', 'रह', 'ऱवस', 'लए', 'लय', 'लकन', 'व', 'वरग', 'वह', 'वह', 'वह', 'वह', 'वल', 'वह', 'व', 'वग़रह', 'सग', 'सकत', 'सकत', 'सबस', 'सभ', 'सथ', 'सबत', 'सभ', 'सर', 'स', 'स', 'ह', 'हआ', 'हई', 'हए', 'ह', 'ह', 'ह', 'हत', 'हत', 'हत', 'हन', 'हन', 'अपन', 'जस', 'हत', 'सभ', 'तह', 'इह', 'दवर', 'इस', 'कह', 'थ', 'उह', 'ओर', 'जह', 'वह', 'अभ', 'बन', 'ह', 'उह', 'उह', 'ह', 'वगरह', 'एस', 'रवस', 'कन', 'नच', 'कफ', 'उस', 'पर', 'भतर', 'ह', 'बह', 'वह', 'कइ', 'यह', 'जह', 'तह', 'कस', 'कइ', 'यह', 'इह', 'जधर', 'इह', 'अद', 'इतयद', 'हइ', 'कनस', 'इसक', 'दसर', 'जह', 'अप', 'कह', 'उनक', 'भ', 'वरग', 'हअ', 'जस', 'नह']
stopwords_en = stopwords.words('english')
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
stopwords_lst = stopwords_hi + stopwords_en + punctuations

In [10]:
df['label_id'] = df.Label.factorize()[0]
import nltk
import re
def processText(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text 
 

In [11]:
for i in range(len(df)):
    df['headline'][i] = processText(df['headline'][i])
count_length()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df.head(10)

,headline,Label,word_count,label_id
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports,19,0
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports,17,0
2,"का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर में ...",Sports,18,0
3,: ऑलराउंडर रोहित गूलिया के दम पर हरियाणा स्ट...,Sports,20,0
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports,13,0
5,: पटना पायरेट्स और बंगाल वॉरियर्स को मिली जी...,Sports,17,0
6,": भारतीय फुटबॉल में कोरोना विस्फोट, ने सप्त...",Sports,17,0
7,'दो चैंपियन' के साथ ट्रेनिंग करते नजर आए शिखर ...,Sports,19,0
8,: जोहानिसबर्ग टेस्ट में पुजारा और रहाणे रहे ...,Sports,21,0
9,कप्तानी विवाद पर पूर्व पाक कप्तान सलमान बट ने ...,Sports,19,0


In [13]:
!pip install indic-nlp-library

     |████████████████████████████████| 40 kB 3.4 MB/s 
     |████████████████████████████████| 2.8 MB 9.2 MB/s 


In [14]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens

In [15]:
df['headline'] = df.headline.apply(lambda x: tokenization(x))
df.head(10)

,headline,Label,word_count,label_id
0,"[मेलबर्न, एयरपोर्ट, पर, घंटों, फंसे, नोवाक, जो...",Sports,19,0
1,"[अर्जेंटीना, के, स्टार, फुटबॉलर, लियोनल, मेसी,...",Sports,17,0
2,"[का, बड़ा, फैसला, ,, कैंपों, और, प्रैक्टिस, से...",Sports,18,0
3,"[:, ऑलराउंडर, रोहित, गूलिया, के, दम, पर, हरिया...",Sports,20,0
4,"[टेनिस, :, टाटा, ओपन, महाराष्ट्र, में, भारत, क...",Sports,13,0
5,"[:, पटना, पायरेट्स, और, बंगाल, वॉरियर्स, को, म...",Sports,17,0
6,"[:, भारतीय, फुटबॉल, में, कोरोना, विस्फोट, ,, न...",Sports,17,0
7,"[', दो, चैंपियन, ', के, साथ, ट्रेनिंग, करते, न...",Sports,19,0
8,"[:, जोहानिसबर्ग, टेस्ट, में, पुजारा, और, रहाणे...",Sports,21,0
9,"[कप्तानी, विवाद, पर, पूर्व, पाक, कप्तान, सलमान...",Sports,19,0


In [16]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (stopwords_lst)]
count_length()
df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,headline,Label,word_count,label_id
0,"[मेलबर्न, एयरपोर्ट, घंटों, फंसे, नोवाक, जोकोवि...",Sports,16,0
1,"[अर्जेंटीना, स्टार, फुटबॉलर, लियोनल, मेसी, कोव...",Sports,14,0
2,"[बड़ा, फैसला, कैंपों, प्रैक्टिस, सेंटर, में, प...",Sports,13,0
3,"[ऑलराउंडर, रोहित, गूलिया, दम, हरियाणा, स्टीलर्...",Sports,13,0
4,"[टेनिस, टाटा, ओपन, महाराष्ट्र, में, भारत, युकी...",Sports,10,0
5,"[पटना, पायरेट्स, बंगाल, वॉरियर्स, मिली, जीत, ट...",Sports,9,0
6,"[भारतीय, फुटबॉल, में, कोरोना, विस्फोट, ने, सप्...",Sports,10,0
7,"[दो, चैंपियन, साथ, ट्रेनिंग, नजर, आए, शिखर, धव...",Sports,15,0
8,"[जोहानिसबर्ग, टेस्ट, में, पुजारा, रहाणे, रहे, ...",Sports,15,0
9,"[कप्तानी, विवाद, पूर्व, पाक, कप्तान, सलमान, बट...",Sports,15,0


In [17]:
df['headline'] = df['headline'].str.join(" ")
df.head(10)

,headline,Label,word_count,label_id
0,मेलबर्न एयरपोर्ट घंटों फंसे नोवाक जोकोविच फिर ...,Sports,16,0
1,अर्जेंटीना स्टार फुटबॉलर लियोनल मेसी कोविड टेस...,Sports,14,0
2,बड़ा फैसला कैंपों प्रैक्टिस सेंटर में पहुंचने ...,Sports,13,0
3,ऑलराउंडर रोहित गूलिया दम हरियाणा स्टीलर्स ने य...,Sports,13,0
4,टेनिस टाटा ओपन महाराष्ट्र में भारत युकी भांबरी...,Sports,10,0
5,पटना पायरेट्स बंगाल वॉरियर्स मिली जीत टाइटंस त...,Sports,9,0
6,भारतीय फुटबॉल में कोरोना विस्फोट ने सप्ताह लिए...,Sports,10,0
7,दो चैंपियन साथ ट्रेनिंग नजर आए शिखर धवन रोहित ...,Sports,15,0
8,जोहानिसबर्ग टेस्ट में पुजारा रहाणे रहे फ्लॉप ट...,Sports,15,0
9,कप्तानी विवाद पूर्व पाक कप्तान सलमान बट ने राह...,Sports,15,0


In [18]:
num_classes = len(df["label_id"].value_counts())

In [19]:
y = tf.keras.utils.to_categorical(df["label_id"].values, num_classes=num_classes)

X_train, X_test, Y_train, Y_test = train_test_split(df['headline'], y, test_size=0.25, random_state=42)

In [20]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
bert_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")

In [21]:
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = bert_preprocess(i)
x = bert_encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

In [22]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
885/885 [==============================] - 720s 790ms/step - loss: 0.5856 - accuracy: 0.7978
Epoch 2/10
885/885 [==============================] - 698s 788ms/step - loss: 0.3709 - accuracy: 0.8531
Epoch 3/10
885/885 [==============================] - 697s 788ms/step - loss: 0.3444 - accuracy: 0.8583
Epoch 4/10
885/885 [==============================] - 697s 788ms/step - loss: 0.3372 - accuracy: 0.8587
Epoch 5/10
885/885 [==============================] - 698s 788ms/step - loss: 0.3283 - accuracy: 0.8617
Epoch 6/10
885/885 [==============================] - 697s 788ms/step - loss: 0.3244 - accuracy: 0.8623
Epoch 7/10
885/885 [==============================] - 697s 788ms/step - loss: 0.3258 - accuracy: 0.8616
Epoch 8/10
885/885 [==============================] - 699s 789ms/step - loss: 0.3238 - accuracy: 0.8613
Epoch 9/10
885/885 [==============================] - 697s 788ms/step - loss: 0.3240 - accuracy: 0.8635
Epoch 10/10
885/885 [==============================] - 697s 788m

In [24]:
accr = model.evaluate(X_test, Y_test)

295/295 [==============================] - 234s 789ms/step - loss: 0.3025 - accuracy: 0.8650


In [25]:
print('Accuracy: {:0.2f}'.format(accr[1]))

Accuracy: 0.86


In [26]:
model.save('/content/gdrive/My Drive/HI_BERT.h5')

In [27]:
model.save('/HI_BERT.h5')

In [28]:
test_classes = []
for i in range(len(Y_test)):
    for j in range(len(Y_test[i])):
        if Y_test[i][j] == 1:
            if j==0:
                test_classes.append(0)
            if j==1:
                test_classes.append(1)
            if j==2:
                test_classes.append(2)
            if j==3:
                test_classes.append(3)
            if j==4:
                test_classes.append(4)

In [29]:
predicted_categories = []
preds = model.predict(X_test)
for i in preds:
    pred_classes = np.argsort(i)[-1:][::-1]
    predicted_categories.append(pred_classes[0])

In [30]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
report = classification_report(test_classes, predicted_categories)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.80      0.86      1908
           1       0.71      0.89      0.79      1873
           2       0.93      0.96      0.94      1825
           3       0.97      0.94      0.95      1940
           4       0.82      0.74      0.78      1890

    accuracy                           0.86      9436
   macro avg       0.87      0.87      0.87      9436
weighted avg       0.87      0.86      0.87      9436



In [31]:
mat = confusion_matrix(test_classes, predicted_categories)
mat

array([[1525,  207,   19,   25,  132],
       [   7, 1668,   68,    0,  130],
       [  12,   42, 1743,   19,    9],
       [  48,    1,   35, 1824,   32],
       [  38,  416,   14,   20, 1402]])

In [32]:
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

In [33]:
sample_text1 = ["सेंसेक्स 1129 पॉइंट्स लुढ़का, रिलायंस का शेयर 4.5% टूटा, मार्केट कैप 2 घंटे में 7.88 लाख करोड़ घटा"]
preds = model.predict(sample_text1)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

business 99.99 %


In [34]:
sample_text2 = ["शादी की खबरों के बीच कटरीना कैफ पर भड़के विक्की कौशल, दोनों के बीच हुआ झगड़ा! जानें वजह"]
preds = model.predict(sample_text2)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

entertainment 99.63 %


In [35]:
sample_text3 = ["पश्चिमी यूपी से पूर्वांचल, अवध से बुंदेलखंड तक, कैसे कमल की फसल को खाद दे रहे पीएम?"]
preds = model.predict(sample_text3)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

politics 93.18 %


In [36]:
sample_text4 = ["भारत पहुंची पाकिस्तान की टीम, 24 नवंबर से भुवनेश्वर में खेला जाएगा वर्ल्ड कपाा"]
preds = model.predict(sample_text4)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

sports 99.53 %


In [37]:
sample_text5 = ["सावधान! अगर ये 7 ऐप्स आपके फोन में हों तो तुरंत करें डिलीट, बन सकते हैं खतरा"]
preds = model.predict(sample_text5)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

tech 99.93 %
